In [1]:
import os, sys

In [2]:
%cd "D:\Schoolwork\TERM 3\WORK\visual_prosody"

D:\Schoolwork\TERM 3\WORK\visual_prosody


In [3]:
import argparse
import yaml
import numpy as np
from pprint import pprint

In [4]:
import argparse
import os

import torch
import yaml
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [39]:
from utils.model import get_model, get_vocoder, get_param_num, vocoder_infer
from utils.tools import to_device, log, synth_one_sample, expand, plot_mel
from model import FastSpeech2Loss
from dataset import Dataset
from utils.auto_tqdm import tqdm

from evaluate import evaluate

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument("--restore_step", type=int, default=0)
parser.add_argument(
    "-p",
    "--preprocess_config",
    type=str,
    required=True,
    help="path to preprocess.yaml",
)
parser.add_argument(
    "-m", "--model_config", type=str, required=True, help="path to model.yaml"
)
parser.add_argument(
    "-t", "--train_config", type=str, required=True, help="path to train.yaml"
)

argString = '-p ./config/Ego4D_final_v6/0628f_preprocess.yaml -m ./config/Ego4D_final_v6/0628f_model.yaml -t ./config/Ego4D_final_v6/0628f_train.yaml'
# args = parser.parse_args()
args = parser.parse_args(argString.split())

In [8]:
pprint(args)

Namespace(restore_step=0, preprocess_config='./config/Ego4D_final_v6/0628f_preprocess.yaml', model_config='./config/Ego4D_final_v6/0628f_model.yaml', train_config='./config/Ego4D_final_v6/0628f_train.yaml')


In [9]:
# Read Config
preprocess_config = yaml.load(
    open(args.preprocess_config, "r"), Loader=yaml.FullLoader
)
model_config = yaml.load(open(args.model_config, "r"), Loader=yaml.FullLoader)
train_config = yaml.load(open(args.train_config, "r"), Loader=yaml.FullLoader)
configs = (preprocess_config, model_config, train_config)

In [10]:
print("Prepare training ...")

preprocess_config, model_config, train_config = configs



Prepare training ...


In [11]:
# Prepare model
model, optimizer = get_model(args, configs, device, train=True)
model = nn.DataParallel(model)
num_param = get_param_num(model)
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)
print("Number of FastSpeech2 Parameters:", num_param)

Number of FastSpeech2 Parameters: 50958465


In [12]:
# Load vocoder
vocoder = get_vocoder(model_config, device)

Removing weight norm...


In [13]:
vocoder

Generator(
  (conv_pre): Conv1d(80, 512, kernel_size=(7,), stride=(1,), padding=(3,))
  (ups): ModuleList(
    (0): ConvTranspose1d(512, 256, kernel_size=(16,), stride=(8,), padding=(4,))
    (1): ConvTranspose1d(256, 128, kernel_size=(16,), stride=(8,), padding=(4,))
    (2): ConvTranspose1d(128, 64, kernel_size=(4,), stride=(2,), padding=(1,))
    (3): ConvTranspose1d(64, 32, kernel_size=(4,), stride=(2,), padding=(1,))
  )
  (resblocks): ModuleList(
    (0): ResBlock(
      (convs1): ModuleList(
        (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
        (2): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(5,), dilation=(5,))
      )
      (convs2): ModuleList(
        (0-2): 3 x Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      )
    )
    (1): ResBlock(
      (convs1): ModuleList(
        (0): Conv1d(256, 256, kernel_size=(7,), stride=(1,), pa

In [14]:
step = args.restore_step + 1

In [15]:
step

1

In [16]:
model.eval()
print()
# message = evaluate(model, step, configs, val_logger, vocoder)


# with open(os.path.join(val_log_path, "log.txt"), "a") as f:
#     f.write(message + "\n")


In [17]:
# evaluate()

In [18]:
dataset = Dataset(
    "val.txt", 'val', preprocess_config, train_config, sort=False, drop_last=False
)

In [19]:
batch_size = train_config["optimizer"]["batch_size"]

In [20]:
loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=dataset.collate_fn,
    )

In [21]:
len(loader)

693

In [22]:
loader.batch_size

4

In [23]:
# Get loss function
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)

In [24]:
import json

In [25]:
synth_one_sample

<function utils.tools.synth_one_sample(targets, predictions, vocoder, model_config, preprocess_config, idx=0)>

In [31]:
os.path.join(train_config['path']['result_path'], 'plots')

'./output/0628f/result/Ego4D_final_v6\\plots'

In [34]:
from scipy.io import wavfile
from matplotlib import pyplot as plt

In [41]:
# => batch:
# return (
#     ids,
#     raw_texts,
#     speakers,
#     texts,
#     text_lens,
#     max(text_lens),
#     mels,
#     mel_lens,
#     max(mel_lens),
#     pitches,
#     energies,
#     durations,
#     speaker_embeddings,
# )
# [12] 对应speaker embedding


# => model output / prediction
# return (
#     output,
#     postnet_output,
#     p_predictions,
#     e_predictions,
#     log_d_predictions,
#     d_rounded,
#     src_masks,
#     mel_masks,
#     src_lens,
#     mel_lens,
# )
output_plot_path = os.path.join(train_config['path']['result_path'], 'plot')
output_mel_syn_path = os.path.join(train_config['path']['result_path'], 'mel', 'syn')
output_mel_gt_path = os.path.join(train_config['path']['result_path'], 'mel', 'gt')
output_wav_syn_path = os.path.join(train_config['path']['result_path'], 'wav', 'synthesized')
output_wav_rec_path = os.path.join(train_config['path']['result_path'], 'wav', 'reconstructed')
os.makedirs(output_plot_path, exist_ok=True)
os.makedirs(output_mel_syn_path, exist_ok=True)
os.makedirs(output_mel_gt_path, exist_ok=True)
os.makedirs(output_wav_syn_path, exist_ok=True)
os.makedirs(output_wav_rec_path, exist_ok=True)

In [44]:
for batchs in loader:
    for targets in batchs:
        targets = to_device(targets, device)
        with torch.no_grad():
            predictions = model(*(targets[2:]))
        basenames = targets[0]
        for i in range(len(predictions[0])):
            basename = basenames[i]
            src_len = predictions[8][i].item()
            mel_len = predictions[9][i].item()
            mel_prediction = predictions[1][i, :mel_len].detach().transpose(0, 1)
            mel_target = targets[6][i, :mel_len].detach().transpose(0, 1)

            torch.save(mel_prediction.cpu(), os.path.join(output_mel_syn_path, f"{basename}.pt"))
            torch.save(mel_target.cpu(), os.path.join(output_mel_gt_path, f"{basename}.pt"))
            
            
            duration = predictions[5][i, :src_len].detach().cpu().numpy()
            if preprocess_config["preprocessing"]["pitch"]["feature"] == "phoneme_level":
                pitch = predictions[2][i, :src_len].detach().cpu().numpy()
                pitch = expand(pitch, duration)
            else:
                pitch = predictions[2][i, :mel_len].detach().cpu().numpy()
            if preprocess_config["preprocessing"]["energy"]["feature"] == "phoneme_level":
                energy = predictions[3][i, :src_len].detach().cpu().numpy()
                energy = expand(energy, duration)
            else:
                energy = predictions[3][i, :mel_len].detach().cpu().numpy()

            with open(os.path.join(preprocess_config["path"]["preprocessed_path"], 
                                   "stats.json")) as f:
                stats = json.load(f)
                stats = stats["pitch"] + stats["energy"][:2]
                                       
            fig = plot_mel(
                [
                    (mel_prediction.cpu().numpy(), pitch, energy),
                    (mel_target.cpu().numpy(), pitch, energy),
                ],
                stats,
                ["Synthetized Spectrogram", "Ground-Truth Spectrogram"],
            )
            ### TODO: change to svg
            plt.savefig(os.path.join(output_plot_path, f"{basename}.png"))
            plt.close()

        # from .model import vocoder_infer

        mel_predictions = predictions[1].transpose(1, 2)
        mel_targets = targets[6].transpose(1, 2)
        
        lengths = predictions[9] * preprocess_config["preprocessing"]["stft"]["hop_length"]
        wav_predictions = vocoder_infer(
            mel_predictions, vocoder, model_config, preprocess_config, lengths=lengths
        )
        wav_targets = vocoder_infer(
        mel_targets, vocoder, model_config, preprocess_config, lengths=lengths
    )
    
        sampling_rate = preprocess_config["preprocessing"]["audio"]["sampling_rate"]
        for wav, basename in zip(wav_predictions, basenames):
            wavfile.write(os.path.join(output_wav_syn_path, f"{basename}.wav"), sampling_rate, wav)
        for wav, basename in zip(wav_targets, basenames):
            wavfile.write(os.path.join(output_wav_rec_path, f"{basename}.wav"), sampling_rate, wav)

        
        break
    break